# Data Preprocessing


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/dacon_cv/ASL
!ls

/content/drive/MyDrive/dacon_cv/ASL
infer.py		MODEL_ZOO.md			 resnext_asl_5.csv
LICENSE			pics				 resnext_asl_75.csv
model_asl		prediction.csv			 resnext.ipynb
model_hard_set		prediction_hard_set_8.csv	 src
model_large_train	prediction_without_cutout_7.csv  train.ipynb
model_new_augmentation	prediction_without_cutout_8.csv  train.py
models			README.md			 tresnet_m.pth
models_hard_asl		requirements.txt		 validate.py
model_without_cutout	resnext_asl


In [3]:
!nvidia-smi

Fri Feb 26 11:02:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
len(os.listdir("./dirty_mnist/"))

KeyboardInterrupt: ignored

In [ ]:
!rm -rf dirty_mnist/ 
!mkdir dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 prediction.csv
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json
 train.json
 val.json


In [ ]:
!unzip -q dirty_mnist_2nd.zip -d dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 prediction.csv
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json
 train.json
 val.json


In [ ]:
!rm -rf dirty_mnist/dirty_mnist_train/

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os
import json

In [ ]:
namelist = os.listdir('./dirty_mnist/')
answer = pd.read_csv('dirty_mnist_answer.csv')
train, val = train_test_split(namelist, test_size=0.1, random_state=23)
train_num = [int(t[:5]) for t in train]
val_num = [int(v[:5]) for v in val]
train_ohe = [True if i in train_num else False for i in range(50000)]
val_ohe = [True if i in val_num else False for i in range(50000)]
train_ans = answer[train_ohe]
val_ans = answer[val_ohe]

In [ ]:
%cd ./dirty_mnist/

In [ ]:
os.makedirs('./dirty_mnist_train', exist_ok=True)
os.makedirs('./dirty_mnist_val', exist_ok=True)
for t in train:
    shutil.move(t, './dirty_mnist_train')
for v in val:
    shutil.move(v, './dirty_mnist_val')

In [ ]:
def csv_to_coco(df, json_name):
    dic = {
        "info": {
            "description": "DACON DIRTY MNIST DATASET",
            "url": "https://oranz.tistory.com/",
            "version": "1.0",
            "year": 2021,
            "contributor": "DACON",
            "date_created": "2021/02/18"
        },
        "licenses": [{
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    for i in range(1, 27):
        dic["categories"].append({"supercategory": "alphabet", "id": i, "name": chr(i + 96)})
        
    anno_id = 1
    for r in range(len(df)):
        image_id = int(df.iloc[r, 0]) # cast to int for JSON serialization
        name = str(image_id)
        name = '0' * (5 - len(name)) + name + '.png'
        dic["images"].append({
            "license": 1,
            "file_name": name,
            "coco_url": "",
            "height": 256,
            "width": 256,
            "date_captured": "2020-05-19 23:03:57",
            "flickr_url": "",
            "id": image_id
        })
        
        for c in range(1, 27):
            if df.iloc[r, c] == 1:
                dic["annotations"].append({
                    "segmentation": [],
                    "area": 232,
                    "iscrowd": 0,
                    "image_id": image_id,
                    "bbox": [1, 5, 4, 9],
                    "category_id": c,
                    "id": anno_id
                })
                anno_id += 1
    with open(json_name, "w") as json_file:
        json.dump(dic, json_file)

In [ ]:
csv_to_coco(val_ans, "val.json")
csv_to_coco(train_ans, "train.json")

# Train


In [7]:
!pip install inplace-abn
!pip install randaugment

     |████████████████████████████████| 143kB 18.5MB/s 
  Created wheel for inplace-abn: filename=inplace_abn-1.1.0-cp37-cp37m-linux_x86_64.whl size=2754632 sha256=d23e688ed0643882bf7266066bd0360695946a4cb053a366c07952f4f25b7f1d
  Stored in directory: /root/.cache/pip/wheels/90/e6/ce/baadcff0441c600caa5874d4d3322a7909e724fb7abab21a15
Successfully built inplace-abn


In [ ]:
cd ./ASL

[Errno 2] No such file or directory: './ASL'
/content/drive/MyDrive/dacon_cv/ASL


In [ ]:
!mkdir model_hard_set
!ls

infer.py		models_hard_asl			 README.md
LICENSE			model_without_cutout		 requirements.txt
model_asl		MODEL_ZOO.md			 src
model_hard_set		pics				 train.ipynb
model_large_train	prediction.csv			 train.py
model_new_augmentation	prediction_without_cutout_7.csv  tresnet_m.pth
models			prediction_without_cutout_8.csv  validate.py


# PPT (모델 구조)

In [8]:
from torchsummary import summary
import easydict
from src.models import create_model

args = easydict.EasyDict({ "data": "../dirty_mnist", 
                          "lr": 23e-5, 
                          "model_name": "tresnet_m", 
                          "model_path": "./tresnet_m.pth",
                          "num_classes": 26, 
                          "workers": 4, 
                          "image_size": 224, 
                          "thre": 0.8,
                          "batch_size": 512,
                          "print_freq": 64
                          })
args.do_bottleneck_head = False

# Setup model
print('creating model...')
model = create_model(args).cuda()
summary(model,(3,256,256))

creating model...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
SpaceToDepthModule-1           [-1, 48, 64, 64]               0
            Conv2d-2           [-1, 64, 64, 64]          27,648
        InPlaceABN-3           [-1, 64, 64, 64]             128
            Conv2d-4           [-1, 64, 64, 64]          36,864
        InPlaceABN-5           [-1, 64, 64, 64]             128
            Conv2d-6           [-1, 64, 64, 64]          36,864
        InPlaceABN-7           [-1, 64, 64, 64]             128
     FastAvgPool2d-8             [-1, 64, 1, 1]               0
            Conv2d-9             [-1, 64, 1, 1]           4,160
             ReLU-10             [-1, 64, 1, 1]               0
           Conv2d-11             [-1, 64, 1, 1]           4,160
          Sigmoid-12             [-1, 64, 1, 1]               0
         SEModule-13           [-1, 64, 64, 64]               0
             ReLU-14 

# Train


In [ ]:
import argparse
import easydict
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data.distributed
import torchvision.transforms as transforms
import os
from torch.optim import lr_scheduler
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.models import create_model
from src.loss_functions.losses import AsymmetricLoss
from randaugment import RandAugment
from torch.cuda.amp import GradScaler, autocast

parser = argparse.ArgumentParser(description='PyTorch MS_COCO Training')
parser.add_argument('data', metavar='DIR', help='path to dataset', default='../dirty_mnist')
parser.add_argument('--lr', default=1e-4, type=float)
parser.add_argument('--model-name', default='tresnet_m')
parser.add_argument('--model-path', default='./tresnet_m.pth', type=str)
parser.add_argument('--num-classes', default=26)
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 16)')
parser.add_argument('--image-size', default=224, type=int,
                    metavar='N', help='input image size (default: 448)')
parser.add_argument('--thre', default=0.8, type=float,
                    metavar='N', help='threshold value')
parser.add_argument('-b', '--batch-size', default=256, type=int,
                    metavar='N', help='mini-batch size (default: 16)')
parser.add_argument('--print-freq', '-p', default=64, type=int,
                    metavar='N', help='print frequency (default: 64)')


def main():
    args = easydict.EasyDict({ "data": "../dirty_mnist", 
                              "lr": 23e-5, 
                              "model_name": "tresnet_m", 
                              "model_path": "./tresnet_m.pth",
                              "num_classes": 26, 
                              "workers": 4, 
                              "image_size": 224, 
                              "thre": 0.8,
                              "batch_size": 512,
                              "print_freq": 64
                              })
    args.do_bottleneck_head = False

    # Setup model
    print('creating model...')
    model = create_model(args).cuda()
    if args.model_path:  # make sure to load pretrained ImageNet model
        state = torch.load(args.model_path, map_location='cpu')
        filtered_dict = {k: v for k, v in state['model'].items() if
                         (k in model.state_dict() and 'head.fc' not in k)}
        model.load_state_dict(filtered_dict, strict=False)
    print('done\n')

    # COCO Data loading
    instances_path_val = os.path.join("/content/drive/My Drive/dacon_cv", 'val.json')
    instances_path_train = os.path.join("/content/drive/My Drive/dacon_cv", 'train.json')
    # data_path_val = args.data
    # data_path_train = args.data
    data_path_val   = f'{args.data}/'    # args.data
    data_path_train = f'{args.data}/'  # args.data
    val_dataset = CocoDetection(data_path_val,
                                instances_path_val,
                                transforms.Compose([
                                    transforms.Resize((args.image_size, args.image_size)),
                                    transforms.ToTensor(),
                                    # normalize, # no need, toTensor does normalization
                                ]))
    train_dataset = CocoDetection(data_path_train,
                                  instances_path_train,
                                  transforms.Compose([
                                      transforms.Resize((args.image_size, args.image_size)),
                                      CutoutPIL(cutout_factor=0.5),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      # normalize,
                                  ]))
    print("len(val_dataset): ", len(val_dataset))
    print("len(train_dataset): ", len(train_dataset))

    # Pytorch Data loader
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=False)

    # Actuall Training
    train_multi_label_coco(model, train_loader, val_loader, args.lr)


def train_multi_label_coco(model, train_loader, val_loader, lr):
    ema = ModelEma(model, 0.9997)  # 0.9997^641=0.82

    # set optimizer
    Epochs = 80
    Stop_epoch = 40
    weight_decay = 1e-4
    criterion = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0.05)
    parameters = add_weight_decay(model, weight_decay)
    optimizer = torch.optim.Adam(params=parameters, lr=lr, weight_decay=0)  # true wd, filter_bias_and_bn
    steps_per_epoch = len(train_loader)
    scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=steps_per_epoch, epochs=Epochs,
                                        pct_start=0.2)

    highest_mAP = 0
    trainInfoList = []
    scaler = GradScaler()
    for epoch in range(Epochs):
        if epoch > Stop_epoch:
          break
        for i, (inputData, target) in enumerate(train_loader):
            inputData = inputData.cuda()
            target = target.cuda()  # (batch,3,num_classes)
            target = target.max(dim=1)[0]
            with autocast():  # mixed precision
                output = model(inputData).float()  # sigmoid will be done in loss !
            loss = criterion(output, target)
            model.zero_grad()

            scaler.scale(loss).backward()
            # loss.backward()

            scaler.step(optimizer)
            scaler.update()
            # optimizer.step()

            scheduler.step()

            ema.update(model)
            # store information
            if i % 50 == 0:
                trainInfoList.append([epoch, i, loss.item()])
                print('Epoch [{}/{}], Step [{}/{}], LR {:.1e}, Loss: {:.1f}'
                      .format(epoch, Epochs, str(i).zfill(3), str(steps_per_epoch).zfill(3),
                              scheduler.get_last_lr()[0], \
                              loss.item()))

        try:
            torch.save(model.state_dict(), os.path.join(
                'model_hard_set/', 'model-{}-{}.pth'.format(epoch + 1, i + 1)))
        except:
            print("model per epoch is not being saved.")
            pass

        model.eval()
        mAP_score = validate_multi(val_loader, model, ema)
        model.train()
        if mAP_score > highest_mAP:
            highest_mAP = mAP_score
            try:
                torch.save(model.state_dict(), os.path.join(
                    'model_hard_set/', 'model-highest.pth'))
            except:
                print("best model is not being saved.")
                pass
        print('current_mAP = {:.2f}, highest_mAP = {:.2f}\n'.format(mAP_score, highest_mAP))


def validate_multi(val_loader, model, ema_model):
    print("starting validation")
    Sig = torch.nn.Sigmoid()
    preds_regular = []
    preds_ema = []
    targets = []
    for i, (input, target) in enumerate(val_loader):
        target = target
        target = target.max(dim=1)[0]
        # compute output
        with torch.no_grad():
            with autocast():
                output_regular = Sig(model(input.cuda())).cpu()
                output_ema = Sig(ema_model.module(input.cuda())).cpu()

        # for mAP calculation
        preds_regular.append(output_regular.cpu().detach())
        preds_ema.append(output_ema.cpu().detach())
        targets.append(target.cpu().detach())

    mAP_score_regular = mAP(torch.cat(targets).numpy(), torch.cat(preds_regular).numpy())
    mAP_score_ema = mAP(torch.cat(targets).numpy(), torch.cat(preds_ema).numpy())
    print("mAP score regular {:.2f}, mAP score EMA {:.2f}".format(mAP_score_regular, mAP_score_ema))
    return max(mAP_score_regular, mAP_score_ema)


main()

creating model...
done

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
loading annotations into memory...
Done (t=2.63s)
creating index...
index created!
len(val_dataset):  5000
len(train_dataset):  45000


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch [0/80], Step [000/088], LR 9.2e-06, Loss: 2355.5
Epoch [0/80], Step [050/088], LR 9.9e-06, Loss: 1945.8
starting validation
mAP score regular 46.59, mAP score EMA 45.93
current_mAP = 46.59, highest_mAP = 46.59

Epoch [1/80], Step [000/088], LR 1.1e-05, Loss: 1818.6
Epoch [1/80], Step [050/088], LR 1.4e-05, Loss: 1797.8
starting validation
mAP score regular 47.10, mAP score EMA 45.95
current_mAP = 47.10, highest_mAP = 47.10

Epoch [2/80], Step [000/088], LR 1.8e-05, Loss: 1783.8
Epoch [2/80], Step [050/088], LR 2.3e-05, Loss: 1766.3
starting validation
mAP score regular 47.88, mAP score EMA 46.00
current_mAP = 47.88, highest_mAP = 47.88

Epoch [3/80], Step [000/088], LR 2.8e-05, Loss: 1764.4
Epoch [3/80], Step [050/088], LR 3.5e-05, Loss: 1752.2
starting validation
mAP score regular 49.13, mAP score EMA 46.05
current_mAP = 49.13, highest_mAP = 49.13

Epoch [4/80], Step [000/088], LR 4.2e-05, Loss: 1751.1
Epoch [4/80], Step [050/088], LR 5.1e-05, Loss: 1751.8
starting validation
mA

# Test


In [ ]:
!rm -rf test_dirty_mnist/
!mkdir test_dirty_mnist/
!unzip -q test_dirty_mnist_2nd.zip -d test_dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [ ]:
import torch
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.loss_functions.losses import AsymmetricLoss, AsymmetricLossOptimized
import torchvision.transforms as transforms
from src.models import create_model
import argparse
import easydict
import numpy as np
import pandas as pd


# parsing args
args = easydict.EasyDict({ "model_path": "/content/drive/My Drive/dacon_cv/ASL/model_hard_set/model-highest.pth",
                          "model_name": "tresnet_m",
                          "workers": 4, 
                          "dataset_type": "MS-COCO", 
                          "th": 0.8,
                          "num_classes": 26
                          })
instances_path_test = '/content/drive/My Drive/dacon_cv/test.json'
data_path_test = '/content/drive/My Drive/dacon_cv/test_dirty_mnist'
test_dataset = CocoDetection(data_path_test,
                            instances_path_test,
                            transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                            ]))

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=4, pin_memory=True, drop_last=False)

predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("/content/drive/My Drive/dacon_cv/sample_submission.csv")
prediction_array = np.zeros([prediction_df.shape[0],
                            prediction_df.shape[1]- 1])


                    
print("test dset: ", test_dataset)

# setup model
print('creating and loading the model...')
state = torch.load(args.model_path, map_location='cpu')
model = create_model(args).cuda()
model.load_state_dict(state, strict=True)
for idx, sample in enumerate(test_data_loader):
    with torch.no_grad():
        # 추론
        model.eval()
        images = sample[0].cuda()
        probs  = model(images)
        probs = probs.cpu().detach().numpy()
        preds = (probs > 0.8)

        # 예측 결과를 
        # prediction_array에 입력
        batch_index = 128 * idx
        prediction_array[batch_index: batch_index + images.shape[0],:] = preds.astype(int)


loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
test dset:  Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /content/drive/My Drive/dacon_cv/test_dirty_mnist
creating and loading the model...


In [ ]:
sample_submission = pd.read_csv("/content/drive/My Drive/dacon_cv/sample_submission.csv")
sample_submission.iloc[:,1:] = prediction_array
sample_submission = sample_submission.astype('int32')
sample_submission.to_csv("prediction_hard_set_8.csv", index = False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1
1,50001,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,0,0,0
2,50002,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1
3,50003,1,1,0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1
4,50004,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0
4996,54996,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4997,54997,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1
4998,54998,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1
